## NHANES Dataset Analysis

This notebook focuses on how to use both the predictive model and the conformance inference model using the National Health and Nutrition Examination Survey (NHANES) dataset. The dataset can be downloaded at https://wwwn.cdc.gov/nchs/nhanes/. 

The dataset has undergone preprocessing steps which includes removing unnecessary variables as well as missing data. More info in the data directory.


In [13]:
import os
import pandas as pd
import random
import numpy as np
import torch
from train import train_conformance_inference
from data_real import load_data

random.seed(0)
torch.manual_seed(0)
np.random.seed(0)


We initialize the main parameters used during the training of the predictive model. The model is training using a `n_fold` cross-validation approach. The number of epochs, batch size, learning rate, weight decay are set to 1000, 64, 0.001, and 0.01, respectively. These values ought to be adjusted in accordance with the specific requirements of the problem, potentially through the application of a grid search technique.

The classification model is trained with an early stop of 4 and no delta. The regression model uses an early stop of 6 with a no delta.


In [2]:
n_folds = 5
n_epochs = 1000
batch_size = 64
learning_rate = 0.001
weight_decay = 0.01
patience_classification = 4
min_delta_classification = 0.0
patience_regression = 6
min_delta_regression = 0.0

In this instance, we employ a distinct combination of input variables, with the diabetes variable serving as the class variable. The NHANES dataset is partitioned into training and test sets in a 75/25 ratio. 

In [3]:
# Clinical measures: body mass index, waist circunference, diastolic and diastolic pressure, pulse, cholesterol
selected_combination = ['RIDAGEYR.x', 'BMXHT', 'BMXWT', 'BMXBMI', 'BMXWAIST', 'BPXDI1', 'BPXSY1', 'BPXPLS', 'LBDSCHSI_43', 'LBXSTR_43', 'LBXSGL_43', 'RIAGENDR', 'LBXGH_39']
file = os.path.join("./real", "data_0.csv")
data = load_data(file, selected_combination, 'diabetes')

# We divide data into training and test splits in a 75-25 ratio.
n = len(data['y'])
indices = random.sample(np.arange(n).tolist(), n)
train_split = indices[:int((3*n) / 4)]
test_split = indices[int((3*n) / 4):]

train_data = {
    'seqn': data['seqn'][train_split],
    'x': data['x'][train_split],
    'y': data['y'][train_split],
    'w': data['w'][train_split],
    'z': data['z'][train_split]
}

We standardize the input data of the training set using the z-score.

In [4]:
# Scale training data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_data['x'] = scaler.fit_transform(train_data['x'])

If needed, the input data of the training set can be balanced, in this case according to the output class variable. 

In [5]:
# Uncomment if data balancing is needed

# # Upsample unbalanced data of the training set
# from sklearn.utils import resample
# y = train_data['y'].reshape(-1)
# diabetic = np.argwhere(y == 1.0).reshape(-1)
# nondiabetic = np.argwhere(y == 0.0).reshape(-1)
# print('Diabetic: {}'.format(diabetic.shape))
# print('Non-diabetic: {}'.format(nondiabetic.shape))
# 
# diabetic_upsample = resample(diabetic, replace=True, n_samples=len(nondiabetic), random_state=42)
# upsampled = np.concatenate([diabetic_upsample, nondiabetic]).reshape(-1)
# print('Upsampled dataset: {}'.format(upsampled.shape))
# np.random.shuffle(upsampled)
# 
# train_data = {
#     'seqn': train_data['seqn'][upsampled],
#     'x': train_data['x'][upsampled],
#     'y': train_data['y'][upsampled],
#     'w': train_data['w'][upsampled],
#     'z': train_data['z'][upsampled]
# }

The input data of the test set are also standardized using the standard scaler previously trained using the training set. 

In [6]:
test_data = {
    'seqn': data['seqn'][test_split],
    'x': data['x'][test_split],
    'o': data['x'][test_split],
    'y': data['y'][test_split],
    'w': data['w'][test_split],
    'z': data['z'][test_split]
}

# Scale test data
test_data['x'] = scaler.transform(test_data['x'])

In the following snippet, we show how to train the conformance inference approach. In this example, we use an alpha value of 0.2. This parameter should be adjusted in accordance with the specific requirements of the problem, potentially through the application of a grid search technique.

In [7]:
# We train the predictor
predictor = train_conformance_inference(
    data=train_data,
    n_folds=n_folds,
    n_epochs=n_epochs,
    batch_size=batch_size,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    alpha=0.2,
    patience_classification=4,
    min_delta_classification=0.0,
    patience_regression=6,
    min_delta_regression=0.0,
    hidden_sizes_bc=[8, 4, 2],
    hidden_sizes_rr=[8, 4, 2]
)

# We test the predictor
ci_result, ci_correct = predictor.classify(test_data)

The conformance inference approach can be used to predict the diabetes risk of patients, although the predictive model is trained with only the 33% of data. Results are returned in the `y` field of the `ci_result` variable. The function `compute_classification_metrics` can be used to compute the area under the curve (`auc`), the accuracy, the recall, precision, f1-score, confusion matrix (`cm`), and cross entropy (`ce`).

In [8]:
from metrics import compute_classification_metrics

compute_classification_metrics(test_data['w'], ci_result['y'], test_data['y'])

{'auc': 0.6851845671524159,
 'accuracy': 0.7297242283821106,
 'recall': 0.41126726330706326,
 'precision': 0.8786853795132236,
 'f1': 0.5602911600696298,
 'cm': [[785.0, 26.0], [287.0, 155.0]],
 'ce': 3.973134961807501}

In the following snippet we show the test dataset concatenated with the results obtained by the predictive model.

In [9]:
import pandas as pd
pd.reset_option('display.max_rows')
pd.set_option('display.max_rows', None)

df = pd.DataFrame(test_data['o'])
df.columns = ['RIDAGEYR.x', 'BMXHT', 'BMXWT', 'BMXBMI', 'BMXWAIST', 'BPXDI1', 'BPXSY1', 'BPXPLS', 'LBDSCHSI_43', 'LBXSTR_43', 'LBXSGL_43', 'RIAGENDR_1', 'RIAGENDR_2', 'LBXGH_39']
df['Seqn'] = test_data['seqn']
df['Prob.'] = ci_result['p']
df['Output'] = ci_result['y']
df['CI'] = ci_result['ci']
df['Target'] = np.int32(test_data['y'])
df

,RIDAGEYR.x,BMXHT,BMXWT,BMXBMI,BMXWAIST,BPXDI1,BPXSY1,BPXPLS,LBDSCHSI_43,LBXSTR_43,LBXSGL_43,RIAGENDR_1,RIAGENDR_2,LBXGH_39,Seqn,Prob.,Output,CI,Target
0,0.433333,0.732659,0.333919,0.175000,0.346888,0.593220,0.272727,0.212121,0.325609,0.031944,0.080702,1.0,0.0,0.128788,80505,0.309019,0,0,0
1,0.316667,0.404624,0.325718,0.279412,0.421577,0.661017,0.233766,0.348485,0.388213,0.064560,0.128070,1.0,0.0,0.204545,62452,0.552795,1,1,0
2,0.333333,0.440751,0.224370,0.173529,0.297925,0.593220,0.337662,0.212121,0.366699,0.087761,0.057895,1.0,0.0,0.113636,65770,0.237935,0,0,0
3,0.266667,0.439306,0.121265,0.079412,0.190041,0.491525,0.246753,0.287879,0.316616,0.035978,0.068421,0.0,1.0,0.098485,76539,0.242834,0,0,0
4,0.083333,0.515896,0.429994,0.326471,0.523651,0.593220,0.415584,0.257576,0.325609,0.168796,0.071930,1.0,0.0,0.128788,79137,0.362700,0,0,0
5,0.533333,0.488439,0.267135,0.195588,0.282158,0.677966,0.337662,0.166667,0.296970,0.031944,0.075439,0.0,1.0,0.166667,75861,0.417757,0,0,1
6,0.133333,0.508671,0.326303,0.239706,0.409959,0.559322,0.298701,0.318182,0.270130,0.055481,0.080702,0.0,1.0,0.106061,78446,0.331855,0,0,0
7,0.433333,0.612717,0.302285,0.185294,0.375934,0.542373,0.233766,0.151515,0.452615,0.043712,0.070175,1.0,0.0,0.106061,74511,0.251058,0,0,0
8,0.616667,0.381503,0.292326,0.257353,0.378423,0.779661,0.584416,0.424242,0.386414,0.018830,0.047368,0.0,1.0,0.143939,78268,0.322781,0,0,0
9,0.616667,0.565029,0.380785,0.266176,0.461411,0.728814,0.337662,0.242424,0.391810,0.184936,0.091228,1.0,0.0,0.143939,77192,0.467837,0,0,0


The predictive model can also be trained independently as follows, in this case, using the complete training set.  

In [10]:
from train_bc import cv_loop_bc
from sklearn.model_selection import KFold
from metrics import compute_classification_metrics

k_fold = KFold(n_splits=n_folds, shuffle=False)
indexes = sorted(range(len(train_data['y']) - 1))
splits = k_fold.split(indexes)

model_ce, metrics_ce = cv_loop_bc(
    data=train_data,
    splits=splits,
    n_epochs=n_epochs,
    batch_size=batch_size,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    patience=patience_classification,
    min_delta=min_delta_classification,
    hidden_sizes=[8, 4, 2]
    # hidden_sizes=[10, 5, 2]
)

The `model_ce` contains the trained model while the `metrics_ce` variable contains the training metrics.  

In [11]:
metrics_ce

{'auc': 0.9159390707013337,
 'accuracy': 0.8221138368953358,
 'recall': 0.675058109027506,
 'precision': 0.8721361051884371,
 'f1': 0.7373665937952125,
 'cm': [[2113.0, 144.0], [431.0, 832.0]],
 'ce': 2.094030328447775}

The following snippet shows how to invoke the predictive model with the test set. Note that `y` field of the test set is not used during the prediction, it is only used for computing the error and performance metrics.  

In [12]:
model_ce.eval()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
with torch.no_grad():
    yp = model_ce(torch.from_numpy(test_data['x']).to(device)).cpu().detach().numpy()

compute_classification_metrics(test_data['w'], yp, test_data['y'])

{'auc': 0.9504048654773973,
 'accuracy': 0.8650681972503662,
 'recall': 0.7901786590087297,
 'precision': 0.8754258175145626,
 'f1': 0.8306207245422282,
 'cm': [[760.0, 51.0], [120.0, 322.0]],
 'ce': 1.4160044193267822}